In [1]:
!pip install geopandas 
!pip install fiona

In [5]:
import geopandas as gpd
import pandas as pd

# 지역 구분 사전
region_mapping = {
    1: "충청남도 공주시",
    2: "충청남도 보령시",
    3: "충청남도 서산시",
    4: "충청남도 논산시",
    5: "충청남도 계룡시",
    6: "충청남도 당진시",
    7: "충청남도 금산군",
    8: "충청남도 부여군",
    9: "충청남도 천안시동남구",
    10: "충청남도 아산시",
    11: "충청남도 서천군",
    12: "충청남도 예산군",
    13: "충청남도 천안시서북구",
    14: "충청남도 청양군",
    15: "충청남도 태안군",
    16: "충청남도 홍성군"
}

# 데이터를 저장할 리스트
data_frames = []

# 파일 읽기 및 데이터 병합
for i in range(1, 17):
    file_path = f'Desktop/충남 토양/{i}/{i}.shp'
    df = gpd.read_file(file_path, encoding='cp949')
    df['지역'] = region_mapping[i]  # 지역 정보 추가
    data_frames.append(df)

# 모든 데이터를 하나로 병합
merged_data = pd.concat(data_frames, ignore_index=True)

# 열 이름 변경
merged_data.rename(columns={
    'SLDPT_TPCD': '토양깊이유형',
    'SCSTX_CD': '토성코드',
    'SLTP_CD': '토양형코드'
}, inplace=True)

# 결측치 제거 및 숫자로 변환
for col in ['토양깊이유형', '토성코드', '토양형코드']:
    merged_data[col] = pd.to_numeric(merged_data[col], errors='coerce')  # 숫자로 변환, 변환 불가능한 값은 NaN 처리

# 결측치 제거
merged_data = merged_data.dropna(subset=['토양깊이유형', '토성코드', '토양형코드'])

# 최빈값 계산
result = []

# 각 지역별 최빈값 계산
for region, group in merged_data.groupby('지역'):
    values = {
        "지역": region,
        "토양깊이유형": group['토양깊이유형'].mode()[0] if not group['토양깊이유형'].mode().empty else None,
        "토성코드": round(group['토성코드'].mean()),
        "토양형코드": round(group['토성코드'].mean())
    }
    result.append(values)

# 결과 데이터프레임 생성
result_df = pd.DataFrame(result)

# 결과 출력
print(result_df)

# 결과를 CSV로 저장
result_df.to_csv('Desktop/충남 토양/충남토양.csv', index=False, encoding='utf-8-sig')


             지역  토양깊이유형  토성코드  토양형코드
0      충청남도 계룡시    20.0     2      2
1      충청남도 공주시    20.0     1      1
2      충청남도 금산군    20.0     2      2
3      충청남도 논산시    20.0     2      2
4      충청남도 당진시    20.0     2      2
5      충청남도 보령시    20.0     2      2
6      충청남도 부여군    20.0     1      1
7      충청남도 서산시    20.0     2      2
8      충청남도 서천군    20.0     2      2
9      충청남도 아산시    20.0     1      1
10     충청남도 예산군    20.0     1      1
11  충청남도 천안시동남구    20.0     1      1
12  충청남도 천안시서북구    20.0     2      2
13     충청남도 청양군    20.0     1      1
14     충청남도 태안군    20.0     1      1
15     충청남도 홍성군    20.0     1      1
